In [ ]:
!pip install opencv-python

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import keras
from sklearn.metrics import f1_score
import cv2
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, Input
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50

In [16]:
datapath = "dataset/"
img_width, img_height = 224,224
classes = ["manipuri", "bharatanatyam", "odissi", "kathakali", "kathak", "sattriya", "kuchipudi", "mohiniyattam"]
class_names_label = {classes:i for i, classes in enumerate(set(classes))}
num_classes = len(classes)
resnet_weights_path = 'resnet_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
# resnet_weights_path = ('https://github.com/fchollet/deep-learning-models/'
#                        'releases/download/v0.2/'
#                        'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [19]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def load_images(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_width, img_height),cv2.INTER_LINEAR)
    return resized

In [4]:
def norm_images(images):
    images = np.array(images,dtype = 'float32') / 255.  
    mean_img = np.mean(images,axos = 0)
    std_img = np.std(images,axis = 0)
    norm_img = (images - mean_img)/std_img
    return norm_img

In [5]:
def load_dataset(folder_name):
#     folders = ["train", "test"]
#     data = []
#     for dataset in folders:
    images = []
    labels = []
    label_file = pd.read_csv(os.path.join(datapath,folder_name+".csv"))
    for file in tqdm(os.listdir(os.path.join(datapath,folder_name))):
        if(folder_name == "train"):
            label = class_names_label[label_file[label_file["Image"] == file]["target"].values[0]]
#                 label = to_categorical(label)
            labels.append(label)
        else:
            labels.append(file)
        image = load_images(os.path.join(datapath,folder_name,file))
        images.append(image)
    norm_img = np.array(images,dtype = 'float32') / 255.  
    
#         labels = np.array(labels, dtype = 'int16')
#     data.append((norm_img,labels))
    return (norm_img,labels)

In [6]:
(x_train, y_train)=load_dataset("train")

(x_test, img_name) = load_dataset("test")

100%|██████████| 156/156 [00:00<00:00, 216.47it/s]


In [7]:
y_train = np.array(y_train, dtype = 'int16')
y_train = to_categorical(y_train)

print(y_train.shape)

(364, 8)


In [26]:
model = Sequential()
model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet',input_tensor=Input(shape=(224,224,3))))
# model.add(Dense(num_classes,activation = 'softmax'))

# model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (128,128,3)))
# #if you resize the image above, change the input shape
# model.add(Convolution2D(32, (3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
# model.add(Convolution2D(64, (3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
# model.add(Convolution2D(128, (3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.layers[0].trainable = False
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 16,392
Non-trainable params: 23,587,712
_________________________________________________________________


In [22]:
early_stops = EarlyStopping(patience=3, monitor='val_accuracy')

In [23]:
model.fit(x_train, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])

Train on 254 samples, validate on 110 samples
Epoch 1/10
254/254 [==============================] - 34s 135ms/step - loss: 2.3983 - accuracy: 0.1260 - val_loss: 2.1564 - val_accuracy: 0.2727
Epoch 2/10


/usr/local/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


254/254 [==============================] - 30s 117ms/step - loss: 1.9646 - accuracy: 0.2480 - val_loss: 2.1518 - val_accuracy: 0.2091
Epoch 3/10
254/254 [==============================] - 30s 120ms/step - loss: 1.5954 - accuracy: 0.4370 - val_loss: 2.1567 - val_accuracy: 0.2455
Epoch 4/10
254/254 [==============================] - 31s 123ms/step - loss: 1.3540 - accuracy: 0.5591 - val_loss: 2.1905 - val_accuracy: 0.2455
Epoch 5/10
254/254 [==============================] - 30s 118ms/step - loss: 1.1325 - accuracy: 0.6732 - val_loss: 2.2673 - val_accuracy: 0.2455
Epoch 6/10
254/254 [==============================] - 36s 140ms/step - loss: 0.9573 - accuracy: 0.7323 - val_loss: 2.3400 - val_accuracy: 0.1727
Epoch 7/10
254/254 [==============================] - 36s 140ms/step - loss: 0.8322 - accuracy: 0.7953 - val_loss: 2.3936 - val_accuracy: 0.1273
Epoch 8/10
254/254 [==============================] - 32s 126ms/step - loss: 0.7239 - accuracy: 0.8307 - val_loss: 2.4037 - val_accuracy: 0.1

In [24]:
predictions = model.predict(x_test)
predictions = np.argmax(predictions,axis = 1)
y_maps = dict()
y_maps = {v:k for k,v in class_names_label.items()}
print(y_maps)
print(predictions)
pred_labels = [y_maps[k] for k in predictions]
print(pred_labels)
sub = pd.DataFrame({'Image':img_name,'target':pred_labels})
sub.to_csv('Solution.csv', index = False)

{0: 'mohiniyattam', 1: 'kathak', 2: 'odissi', 3: 'sattriya', 4: 'manipuri', 5: 'kathakali', 6: 'kuchipudi', 7: 'bharatanatyam'}
[6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 6 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 6 7 7 7 7 6 7
 7 6 7 7 6 7 7 7 7 7 7 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 6 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 6 7 7 7 7 7 7 7 6 7 6 7 6 7 7 7 7 6 6 7 7 6 6 7 7 7 7
 7 7 7 7 7 6 6 7]
['kuchipudi', 'kuchipudi', 'kuchipudi', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'bha